In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title
from bs4 import BeautifulSoup as bs
import requests

house_title_list = []

driver = webdriver.Chrome(r'/Users/mohamedsellamia/airbnb/chromedriver')


now = datetime.datetime.now() #for csv file title

#For csv file names
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

#Open CSV Writer
filename = paste(['airBnb_To'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)

index = 1

house_all_links = []




items_offset = 1
for adults in range (1, 20):
    for items_offset in range (1, 289):
        link = "https://www.airbnb.com/s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_type=pagination&place_id=ChIJpTvG15DL1IkRd8S0KlBVNTI&screen_size=large&hide_dates_and_guests_filters=false&s_tag=sTnSJbXh&section_offset=4&items_offset="+str(items_offset)+"&adults="+str(adults)
        house_all_links.append(link)
        
print("Total number of potential match: ", len(house_all_links))

done = 1
for link in house_all_links: 

    page = requests.get(link)
    soup = bs(page.text, "lxml")
    match = soup.find_all('a', href=True)
    link_list = []
    for i in match:
        link = i["href"]
        link_list.append(link)  
    for url in link_list[2:3]:
        url = ("https://www.airbnb.ca"+str(url))
        airbnb_dict = {}
        driver.get(url)
        time.sleep(7)

        try:
            house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
        except:
            house_title = None
        try:
            house_type =  driver.find_element_by_xpath('//*[@id="room"]/div[2]/div/div[2]/div[1]/div/div[3]/div/div/div/div[3]/div/div[2]/div[1]/span').text
        except:
            house_type = None
        try:
            house_location =  driver.find_element_by_xpath('//div[@class="_czm8crp"]').text
        except:
            house_location = None

        try:
            price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
        except:
            price = None        

        try:
            no_of_reviews = driver.find_element_by_xpath('//div[@class="_14hveccc"]').text
        except:
            no_of_reviews = None  

        try:
            rating = driver.find_element_by_xpath('//div[@class="_10za72m2"]').text

        except:
            rating = None

        try:
            max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')
            house_detail = [detail.text for detail in max_occupancy ]

        except:

            house_detail = None
            
        try:
            amenities = driver.find_elements_by_xpath('//*[@id="amenities"]//div[@class ="_czm8crp"]')
            amenity_detail = [amenity.text for amenity in amenities]

        except:

            house_detail = None
        if house_title not in house_title_list:
            house_title_list.append(house_title)
            print ([house_title, house_type, house_location, price, no_of_reviews, rating, house_detail,amenity_detail])
            
            airbnb_dict['house_title'] = house_title
            airbnb_dict['house_type'] = house_type
            airbnb_dict['house_locatione'] = house_location
            airbnb_dict['price'] = price
            airbnb_dict['no_of_reviews'] = no_of_reviews
            airbnb_dict['rating'] = rating
            airbnb_dict['house_detail'] = house_detail
            airbnb_dict['amenity_detail'] = amenity_detail
            writer.writerow(airbnb_dict.values())
            
            print ("units parsed: ", done, "units left: ", len(house_all_links) - done)
        
    done = done + 1
    
csv_file.close()
driver.close()


Total number of potential match:  5472
['*City View Studio 3 mins to University and Subway*', 'Entire home', 'Toronto', '$99', '41 reviews', '4.95', ['3 guests', 'Studio', '2 beds', '1 bath', '\U000f0004\n41 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Hot tub', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  1 units left:  5471
['High Floor | Balcony | Fast WiFi | ❤️ of Downtown', 'Entire home', 'Toronto', '$90', '316 reviews', '4.85', ['2 guests', 'Studio', '1 bed', '1 bath', '\U000f0004\n480 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Hot tub', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  2 units left:  5470
['Cozy, Private Studio Flat 21 min to downtown by GO', 'Entire home', 'Toronto', None, None, '4.89', ['2 guests', 'Studio', '1 bed', '1 bath', '\U000f0004\n731 Reviews', '\U000f1101\n2 References', '\U000f0019\nVerified'], ['Free parking on pr

['Luxurious penthouse duplex - Downtown Central', 'Entire home', 'Toronto', '$118', '251 reviews', '4.29', ['8 guests', '1 bedroom', '4 beds', '1 bath', '\U000f0004\n7991 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Laptop-friendly workspace', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  31 units left:  5441
['Luxury 2bd+2bth in the Heart of Downtown!', 'Entire home', 'Toronto', '$105', '308 reviews', '4.87', ['5 guests', '2 bedrooms', '3 beds', '2 baths', '\U000f0004\n308 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Laptop-friendly workspace', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  32 units left:  5440
['Studio Symmetric', 'Entire home', 'Toronto', '$74', '368 reviews', '4.48', ['6 guests', 'Studio', '3 beds', '1 bath', '\U000f0004\n7991 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'Laptop-friendly workspace', 'Iron', 'All hosts mu

['Cozy Room in a Condo', 'Sparkling clean', 'Toronto', '$26', '176 reviews', '4.67', ['2 guests', '1 bedroom', '1 bed', '1 shared bath', '\U000f0004\n271 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Elevator', 'Indoor fireplace', 'Wifi', 'Unavailable: Carbon monoxide detector\nCarbon monoxide detector', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  60 units left:  5412
['Renovated DOWNTOWN Studio + PATIO Queen Street W', 'Entire home', 'Toronto', '$49', '139 reviews', '4.65', ['3 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n992 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'Laptop-friendly workspace', 'TV', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  61 units left:  5411
['Modern and cute suit Close to Lake ON', 'Entire home', 'Toronto', '$99', '478 reviews', '4.68', ['2 guests', 'Studio', '1 bed', '1 bath', '\U000f0004\n502 Reviews', '\U000f0019\nVerified

['Spacious 2 bedroom, Little Portugal', 'Entire home', 'Toronto', '$60', '223 reviews', '4.89', ['5 guests', '2 bedrooms', '3 beds', '2 baths', '\U000f0004\n328 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'Cable TV', 'Laptop-friendly workspace', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  101 units left:  5371
['Charming Queen St. Apartment', 'Entire home', 'Toronto', '$75', '281 reviews', '4.67', ['2 guests', 'Studio', '1 bed', '1 bath', '\U000f0004\n578 Reviews', '\U000f0019\nVerified'], ['Wifi', 'Kitchen', 'Cable TV', 'Laptop-friendly workspace', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  103 units left:  5369
["Cozy Bedroom in Toronto's Old Town", 'Mercedeh is a Superhost', 'Toronto', '$60', '325 reviews', '4.98', ['1 guest', '1 bedroom', '1 bed', '1 private bath', '\U000f0004\n325 Reviews'], ['Indoor fireplace', 'Breakfast', 'Wifi', 'Laptop-friendly workspace', 'All hosts mu

['Perfectly Located by CN Tower & Rogers Centre', 'Entire home', 'Toronto', '$85', '123 reviews', '4.72', ['3 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n1557 Reviews', '\U000f1101\n2 References', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Hot tub', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  129 units left:  5343
['Room Cozy Red - Little Italy', 'Sparkling clean', 'Toronto', '$39', '232 reviews', '4.21', ['4 guests', '1 bedroom', '2 beds', '1 shared bath', '\U000f0004\n7991 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Wifi', 'Kitchen', 'Laptop-friendly workspace', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  131 units left:  5341
['(A1)CN Tower, MTCC, ACC, Rogers Center', 'Entire home', 'Toronto', '$94', '138 reviews', '4.78', ['3 guests', '1 bedroom', '2 beds', '1 bath', '\U000f0004\n2117 Reviews', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'K

['Modern Chic Suite by CN Tower with City View', 'Entire home', 'Toronto', '$100', '101 reviews', '4.64', ['3 guests', '1 bedroom', '1 bed', '1 bath', '\U000f0004\n1565 Reviews', '\U000f1101\n2 References', '\U000f0019\nVerified'], ['Elevator', 'Wifi', 'Kitchen', 'Hot tub', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  5428 units left:  44
['3 BDRM, 2 Bathroom Suite with Parking', 'Entire home', 'Toronto', None, None, '4.73', ['8 guests', '3 bedrooms', '4 beds', '2 baths', '\U000f0004\n163 Reviews', '\U000f0019\nVerified'], ['Free parking on premises', 'Elevator', 'Wifi', 'Kitchen', 'All hosts must meet our requirements and maintain a high overall rating.']]
units parsed:  5430 units left:  42
